In [1]:
%pip install pyserial

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from IPython.display import clear_output
import serial
import time
import math

In [3]:
def setVoltageType(voltage_type):
    global voltage_correction_factor
    if voltage_type == 0:
        voltage_correction_factor = 0.5769
    elif voltage_type == 1:
        voltage_correction_factor = 0.5061
    elif voltage_type == 2:
        voltage_correction_factor = 0.3151
    elif voltage_type == 3:
        voltage_correction_factor = 0.1995
    else:
        voltage_correction_factor = 0.5769
        print("Invalid voltage type. Defaulting to 0.5769")

def getCorrectedVoltage(voltage):
    global voltage_correction_factor
    return (voltage - 1650) * voltage_correction_factor

In [4]:
def setCurrentType(current_type):
    global current_correction_factor
    if current_type == 0:
        current_correction_factor = 0.866
    elif current_type == 1:
        current_correction_factor = 0.642
    elif current_type == 2:
        current_correction_factor = 0.403
    elif current_type == 3:
        current_correction_factor = 0.255
    elif current_type == 4:
        current_correction_factor = 0.173
    elif current_type == 5:
        current_correction_factor = 0.096
    elif current_type == 6:
        current_correction_factor = 0.064
    elif current_type == 7:
        current_correction_factor = 0.096
    elif current_type == 8:
        current_correction_factor = 0.0265
    else:
        current_correction_factor = 0.866
        print("Invalid current type. Defaulting to 0.866")

def getCorrectedCurrent(current):
    global voltage_correction_factor
    return (current - 1650) * current_correction_factor

In [5]:
SYSTEM_TYPE =           0   # Change this value according to the schematic "Type Of Electrical System" table
CURRENT_TYPE =          8   # Change this value according to the schematic "Type Of Current" table
VOLTAGE_TYPE =          2   # Change this value according to the schematic "Type Of Voltage" table

setVoltageType(VOLTAGE_TYPE)
setCurrentType(CURRENT_TYPE)

In [23]:
power_vision = serial.Serial(
    port='COM21',
    baudrate=115200,
)

time_to_analyze = 20

frequency = 0
voltage_a_array = []
current_a_array = []
start_time = time.time()
actual_time = start_time
while (actual_time - start_time) < time_to_analyze:
    try:
        data = power_vision.readline().decode().strip()
        if data:
            data = data.split(' ')
            if data[0] == "samples:":
                voltage_a_array.append(getCorrectedVoltage(float(data[1])))
                current_a_array.append(getCorrectedCurrent(float(data[2])))
            elif data[0] == "frequency:":
                frequency = float(data[1])
                if frequency > 70:
                    continue
                
                samples_voltage_a = voltage_a_array.copy()
                samples_current_a = current_a_array.copy()
                voltage_a_array = []
                current_a_array = []

                number_of_samples = len(samples_voltage_a)
                print(number_of_samples)

                clear_output(wait=True)
    except:
        pass

    actual_time = time.time()

power_vision.close()

23


In [22]:
power_vision.close()